# Examples of Multicore Applications:
# Shared-Memory Multiprocess Applications
In this notebook we look at multiprocess applications in IoTPy. The processes share memory. Associated with each process is an agent. The application can also have: 
<ol>
    <li> source threads that acquire data from external sources and </li>
    <li> actuator threads the get data from output queues. </li>
</ol>
<b> The central idea is that an application is specified by connecting outputs of processes to inputs of processes.</b>

## The Agent associated with a Process
A process in a multicore application executes an agent with the following signature:
<br>
<br>
<b>f(in_streams, out_streams)</b>
<br>
<br>
where:
<ol>
    <li> <i>f</i> is a function. </li>
    <li> <i>in_streams</i> is a list of input streams. </li>
    <li> <i>out_streams</i> is a list of output streams. </li>
</ol>

In [1]:
import sys
import time
import threading

sys.path.append("../")
from IoTPy.core.stream import Stream, StreamArray, run
from IoTPy.agent_types.op import map_element, map_list, map_window
from IoTPy.helper_functions.recent_values import recent_values
from IoTPy.helper_functions.print_stream import print_stream

from IoTPy.concurrency.multicore import get_processes, get_processes_and_procs
from IoTPy.concurrency.multicore import terminate_stream
from IoTPy.concurrency.multicore import get_proc_that_inputs_source
from IoTPy.concurrency.multicore import extend_stream

Next we show a collection of agents, <i>f</i>, <i>g</i>, <i>h</i>, and <i>r</i>, with this signature. We will use these agents in the examples of multicore programs.

In [2]:
def f(in_streams, out_streams):
    map_element(lambda v: v+100, in_streams[0], out_streams[0])

def g(in_streams, out_streams):
    s = Stream('s')
    map_element(lambda v: v*2, in_streams[0], s)
    print_stream(s, 's')

def h(in_streams, out_streams):
    map_element(lambda v: v*2, in_streams[0], out_streams[0])

def r(in_streams, out_streams):
    t = Stream('t')
    map_element(lambda v: v*3, in_streams[0], t)
    print_stream(t, 't')

## Threads
A process may execute an arbitrary number of threads. You can use any thread target. 
<br>
<br>
Most threads in IoTPY applications pass data to the application or get data from the application. A thread that passes data from an external source, such as a sensor or a Twitter stream, to an IoTPy process is called a <b>source thread</b>. 
### Source Threads
A source thread calls the following function to put data into a stream:
<br>
<br>
<b>extend_stream(procs, data, stream_name)</b>
<br>
<br>
where
<ol>
    <li> <i>procs</i> is a list of process metadata created from the specification of a multicore program. <i>procs</i> is passed as a parameter to the thread target. We will discuss <i>procs</i> later. </li>
    <li> <i>data</i> is a list or an array. </li>
    <li> <i>stream_name</i> is a string which is the name of a stream.</li>
</ol>
In the example, <i>source_thread_target</i>, the function has a single argument <i>procs</i>. All thread targets that extend streams must have <i>procs</i> as one of its arguments.
<br>
<br>
This function executes a loop in which puts specified data into a stream called <i>x</i> and then sleeps thus yielding the thread.
<br>
<br>
<b>terminate_stream</b>

### Sources
A source in a multiprocess application is associated with a process. A source <i>s</i> in a process <i>p</i> is essentially an output stream of <i>p</i>; it differs from output streams created by <i>p</i> in the sense that it is fed by a thread rather than computed by <i>p</i>. However, we don't include <i>s</i> in the list of <b>outputs</b> of <i>p</i>; instead we include <i>s</i> in the list of <b>sources</b> of <i>p</i>.
<br>
<br>
Note that a source thread that generates a source <i>s</i> in a process <i>p</i> can run in a different process <i>p'</i>. You want to choose the process in which to run a thread to balance the computational load across processes. If the output of a source <i>s</i> feeds input streams of exactly one process <i>r</i> then efficiency suggests that <i>s</i> should be a source of <i>r</i>; however, you can make <i>s</i> the source of any process.


## Steps to create a multiprocess application
You may find the following steps helpful in creating a multiprocess application. You don't have to follow exactly these steps in this order.
<br>
<br>
<i>Step 0</i>: <b>Define agent functions and source thread targets.</b>
<ol>
        <li> <i>Step 0.0</i>: Each process has an agent associated with it, as described earlier. Specify the agent functions for each process. Recall that an agent function has the form:
<br>
            <b>f(in_streams, out_streams)</b>.</li>
    <li><i>Step 0.1 </i> Define the thread targets for each source. These thread targets typically extend a source stream, and finally terminate the stream.</li>
</ol>
<i>Step 1: </i> <b>Give the multicore_specification of streams and processes.</b> The multicore specification specifies a list of streams and a list of agents.
<br>
<i>Step 2: </i> <b>Create processes</b> by calling:
<br>
processes, procs = get_processes_and_procs(multicore_specification)
<br>
<i>Step 3: </i> <b>Create threads</b> (if any). An example creation of a thread is:
<br>
thread_0 = threading.Thread(target=source_thread_target, args=(procs,))
<br>
<i>Step 4: </i> <b>Specify which process each thread runs in.</b> An example:
<br>
procs['p1'].threads = [thread_0]
<br>
<i>Step 5: </i>. <b>Start, join and terminate processes </b> by calling
<br>
for process in processes: process.start()
for process in processes: process.join()
for process in processes: process.terminate()
    

In [3]:
# Step 0: Define source thread target (if any).
# We will use this thread target for the next few examples.
def source_thread_target(procs):
    for i in range(3):
        extend_stream(procs, data=list(range(i*2, (i+1)*2)), stream_name='x')
        time.sleep(0.001)
    terminate_stream(procs, stream_name='x')

## Simple example of a multicore program
![alt text](ExamplesOfSimpleMultipleProcess.jpg "Simple MultiProcess")

### Multicore specification: Processes and their connecting streams
Look at <b>multicore_specification</b>. The specification states that the program has two processes called p0 and p1. Process p0 has a single input stream <i>x</i> and a single output stream <i>y</i>. Process p1 has a single input stream <i>y</i> and no output streams. Thus, the output <i>y</i> of process p0 is the input of process p1.
<br>
### Multicore specification: Streams
Streams are specified by a list of pairs where each pair is a stream name and a stream type. The stream type 'i' identifies integers, 'f' floats and 'd' double. We use stream types to allow processes to share memory in Python 2.7+. In this example, the pair ('x', 'i') says that the program has a stream <i>x</i> of type int.
<br>
### Multicore specification: Sources
Process p0 has a <b>source_functions</b> called <i>h</i>. Function <i>h</i> executes in its own thread within process p0; this thread is started when the process is started. Function <i>h</i> has a single argument called <i>proc</i> which is a dummy argument that represents a process. 
<br>
<br>
Function <i>h</i> puts data into stream <i>x</i> when it executes <b>proc.copy_stream()</b>. The thread executing <i>h</i> then sleeps for 0.001 seconds before appending more data to stream <i>x</i>. Finally, the thread signals that the source has terminated appending data to stream <i>x</i> by calling <b>proc.finished_source('x')</b>.
### Process Structure
The source <i>h</i> outputs a stream <i>x</i> which is an input of process p0. The output <i>y</i> of process p0 is an input to process p1.
### Process Computations
The computation of a process is specified by a function with two arguments <i>in_streams</i> and <i>out_streams</i>. The computation carried out by p0 is specified by function <i>f</i> which reads a single input stream, <i>in_streams[0]</i> and write a single output stream, <i>out_streams[0]</i>. This agent makes:
<br>
<br>
<b> y[n] = x[n] + 100 </b>
<br>
<br>
The computation carried out by process p1 is specified by function <i>g</i> which prints <b>2 * y[n]</b> for all n.
<br>
<br>
The source function <i>h</i> sets x[n] to n, and so this multicore process prints:
<br>
<br>
<b> 2 * (n + 100) </b>

In [4]:
# Step 1: multicore_specification of streams and processes.
multicore_specification = [
    # Streams
    [('x', 'i'), ('y', 'i')],
    # Processes
    [{'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y'], 'sources':['x']},
     {'name': 'p1', 'agent': g, 'inputs': ['y']}]]

# Step 2: Create processes.
processes, procs = get_processes_and_procs(multicore_specification)

# Step 3: Create threads (if any).
thread_0 = threading.Thread(target=source_thread_target, args=(procs,))

# Step 4: Specify which process each thread (if any) runs in.
# thread_0 runs in the process called 'p1'
procs['p1'].threads = [thread_0]

# Step 5: Start, join and terminate processes.
for process in processes: process.start()
for process in processes: process.join()
for process in processes: process.terminate()

AttributeError: Can't pickle local object 'MulticoreProcess.make_process.<locals>.target'

In [ ]:
# Step 1: multicore_specification of streams and processes.
multicore_specification = [
    # Streams
    [('x', 'i'), ('y', 'i')],
    # Processes
    [{'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y']},
     {'name': 'p1', 'agent': g, 'inputs': ['y'], 'sources':['x']}]]

# Step 2: Create processes.
processes, procs = get_processes_and_procs(multicore_specification)

# Step 3: Create threads (if any).
thread_0 = threading.Thread(target=source_thread_target, args=(procs,))

# Step 4: Specify which process each thread (if any) runs in.
# thread_0 runs in the process called 'p1'
procs['p1'].threads = [thread_0]

# Step 5: Start, join and terminate processes.
for process in processes: process.start()
for process in processes: process.join()
for process in processes: process.terminate()

## Example of Three Processes in a Row
![alt text](ThreeProcessesInARow.jpg "Title")
This example is the same as the previous one except that it has a third process attached to process p2. The source thread <i>h</i> feeds stream <i>x</i> which is the input to process p0. The output of p0 is stream <i>y</i> which is the input to process p1. The output of p1 is stream <i>z</i> which is the input to process p2.
<br>
### Streams
[('x', 'i'), ('y', 'i'), ('z', 'i')]
This specifies that this system has three streams called 'x', 'y' and 'z' which contain ints.
### Sources
<b>Source Function</b> <i>h</i>
<br>
This function runs in its own thread. The function puts [0, 1, 2] into the stream called <i>x</i>, then sleeps, and then puts [3, 4, 5] into <i>x</i>. The function then calls <i>finished_source</i> to indicate that it has finished executing and so no further values will be appended to <i>x</i>.
<br>
<br>
This function executes in a thread that runs in process <i>p0</i> because <i>h</i> appears in the specification for <i>p0</i>:
<br>
{'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y'], 'sources': ['x'], <b>'source_functions':[h]</b>}
<br>
<b>Stream Sources</b> Stream <i>x</i> is a source in process <i>p0</i> because it appears in the specification of process <i>p0</i>.
### Process Structure
<ol>
    <li>Source function <i>h</i> feeds stream <i>x</i> which is an input of process <i>p0</i>. </li>
    <li> Output stream <i>y</i> of process <i>p0</i> is an input stream of process <i>p1</i>.</li>
    <li> Output stream <i>z</i> of process <i>p1</i> is an input stream of process <i>p2</i>.</li>
    <li> Process <i>p2</i> has no output streams. </li>
</ol>

### Process Functions
Each process function has parameters <i>in_streams</i>, <i>out_streams</i> and possibly additional keyword or positional arguments. The process functions associated with processes <i>p0</i>, <i>p1</i>, and <i>p2</i> are <i>f</i>, <i>g</i> and <i>r</i>, respectively. The process function for a process is in the processes part of <i>multicore_specification</i>.
<br>
<ol>
    <li> The source extends stream <i>x</i> with [0, 1, 2, 3, 4, 5] and then calls <i>finished_source</i>. Thus <b>x[n] = n </b> for n less than 6. </li>
    <li> Process function <i>f</i> of <i>p0</i> adds 100 to its <i>in_streams[0]</i> which is stream <i>x</i> and puts the result in its <i>out_streams[0]</i> which is stream <i>y</i>. Thus <b>y[n] = x[n]+100 = n + 100 for </b> </li>.
    <li> Process function <i>g</i> of <i>p1</i> multiplies 2 to its <i>in_streams[0]</i> which is stream <i>y</i> and puts the result in its <i>out_streams[0]</i> which is stream <i>z</i>. Thus <b>z[n] = 2*y[n] = 2n + 200 for </b> </li>.
    <li> Process function <i>r</i> of <i>p2</i> creates a stream <i>s</i> and multiplies 3 to its <i>in_streams[0]</i> which is stream <i>z</i> and and puts the result stream <i>s</i>. This function also prints stream <i>s</i>. Thus it prints <b>3*z[n] = 6n + 600 for </b> </li>.
</ol>

In [ ]:
# Step 1: multicore_specification of streams and processes.
multicore_specification = [
    # Streams
    [('x', 'i'), ('y', 'i'), ('z', 'i')],
    # Processes
    [{'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y']},
     {'name': 'p1', 'agent': h, 'inputs': ['y'], 'outputs': ['z'], 'sources': ['x']},
     {'name': 'p2', 'agent': r, 'inputs': ['z']}]
    ]

# Step 2: Create processes.
processes, procs = get_processes_and_procs(multicore_specification)

# Step 3: Create threads (if any)
thread_0 = threading.Thread(target=source_thread_target, args=(procs,))

# Step 4: Specify which process each thread runs in.
# thread_0 runs in the process called 'p1'
procs['p1'].threads = [thread_0]

# Step 5: Start, join and terminate processes.
for process in processes: process.start()
for process in processes: process.join()
for process in processes: process.terminate()

## Example of Multicore with NumPy Arrays
This example illustrates the use of <b>StreamArray</b> which is a stream treated as a NumPy array with an arbitrarily large number of rows. Using <i>StreamArray</i> can be more efficient than using <i>Stream</i> for large computations. 
<br>
<br>
These examples are simple and small; however, in most applications each process function would convert an input stream to a <i>StreamArray</i> and carry out a lot computation as arrays before sending the results as output streams.
<br>
<br>
The streams, sources, and process structure are similar to the previous two examples. The process functions differ in that the functions in this example use <i>StreamArray</i> whereas the earlier examples used <i>Stream</i>.
<br>
<br>
You convert a Stream of numbers to a StreamArray of ints, floats, or doubles by calling the functions <b>dtype_int</b>, <b>dtype_float</b>, and <b>dtype_double</b> respectively.
<br>
<br>
In this example, the agent functions <i>f</i> and <i>g</i> operate on StreamArrays of floats though the source function <i>h</i> generates a stream of int.
![alt text](ExampleOfTwoNumpyAgents.jpg "Example of Two Numpy Agents")

In [ ]:
import numpy as np
from IoTPy.helper_functions.type import dtype_float

def test_multicore_with_arrays():
    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agent functions.

    # f_numpy is the agent function for processor called 'p0'.
    def f_numpy(in_streams, out_streams):
        map_window(
            np.mean, dtype_float(in_streams[0]), out_streams[0],
            window_size=2, step_size=2)

    # g_numpy is the agent function for processor called 'p1'.
    def g_numpy(in_streams, out_streams):
        t = StreamArray('t')
        map_window(max, dtype_float(in_streams[0]), t, 
                   window_size=2, step_size=2)
        print_stream(t)

    # Step 0.1: Define source thread targets (if any).
    def thread_target_numpy(procs):
        for i in range(3):
            extend_stream(procs, data=list(range(i*10, (i+1)*10)), 
                          stream_name='x')
            # Sleep to simulate an external data source.
            time.sleep(0.001)
        # Terminate stream because this stream will not be extended.
        terminate_stream(procs, stream_name='x')

    # Step 1: multicore_specification of streams and processes.
    # Specify Streams: list of pairs (stream_name, stream_type).
    # Specify Processes: name, agent function, 
    #       lists of inputs and outputs, additional arguments.
    multicore_specification = [
        # Streams
        [('x', 'i'), ('y', 'f')],
        # Processes
        [{'name': 'p0', 'agent': f_numpy, 'inputs':['x'], 
          'outputs': ['y'], 'sources': ['x']},
         {'name': 'p1', 'agent': g_numpy, 'inputs': ['y']}]
    ]

    # Step 2: Create processes.
    processes, procs = get_processes_and_procs(multicore_specification)

    # Step 3: Create threads (if any)
    thread_0 = threading.Thread(target=thread_target_numpy, args=(procs,))

    # Step 4: Specify which process each thread runs in.
    # thread_0 runs in the process called 'p1'
    procs['p1'].threads = [thread_0]

    # Step 5: Start, join and terminate processes.
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

test_multicore_with_arrays()

## Example of Merging Streams from Multiple Processes
This example shows a slightly more complex process structure. The example has four processes
called <i>coordinator</i>, <i>sine</i>, <i>cosine</i>, and <i>tangent</i>. The <i>coordinator</i> generates a sequence of values that are sent to other processes which compute sines, cosines and tangents of these values and send the results back to the <i>coordinator</i>. The <i>coordinator</i> then computes the square of the error --- the difference between tangent and sine/cosine.
<br>
<br>
This example gives names to agents. This is helpful in debugging because the error statements identify the agent that caused the error. We haven't given names to agents in some examples for brevity.
![alt text](ExampleOfCoordinator.jpg "Example of Coordinator")

### Process Structure
<ol>
    <li> A source function <i>h</i> extends stream <i>x</i> with a sequence of 10 values between 0.0 and pi. This source function executes in a thread in the process called <i>coordinator</i>. Stream <i>x</i> is an input for all processes.
    </li> 
    <li> Agents <i>sine</i>, <i>cosine</i>, and <i>tangent</i> read stream <i>x</i> and output streams <i>sines</i>, <i>cosines</i>, and <i>tangents</i> respectively. These streams are inputs to process <i>coordinate</i>.
    </li>
<ol> 

In [ ]:
from IoTPy.agent_types.merge import zip_map

def example_merging_streams_from_multiple_processes():
    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agent functions.
    
    # sine is the agent function for the process called 'sine'.
    def sine(in_streams, out_streams):
        map_element(np.sin, dtype_float(in_streams[0]), out_streams[0], 
                    name='sine')

    # cosine is the agent function for the process called 'cosine'.
    def cosine(in_streams, out_streams):
        map_element(np.cos, dtype_float(in_streams[0]), out_streams[0], 
                    name='cosine')

    # tangent is the agent function for the process called 'tangent'.
    def tangent(in_streams, out_streams):
        map_element(np.tan, dtype_float(in_streams[0]), out_streams[0], 
                    name='tangent')

    # coordinate is the agent function for the process called 'coordinate'.
    def coordinate(in_streams, out_streams):
        x, sines, cosines, tangents = in_streams

        def f(lst): return lst[0]/lst[1]

        def g(lst):
            error_squared= (lst[0] - lst[1])**2
            return error_squared
    
        ratios = Stream('ratios')
        errors = Stream('errors')
        zip_map(f, [sines, cosines], ratios, name='sine / cosine')
        zip_map(g, [ratios, tangents], errors, name='compute error')
        print_stream(errors, 'error')

    # # Step 0.1: Define source thread target (if any).
    def source_thread_target(procs):
        extend_stream(procs, data = np.linspace(0.0, np.pi, 10), stream_name='x')
        terminate_stream(procs, stream_name='x')

    # Step 1: multicore_specification of streams and processes.
    # Specify Streams: list of pairs (stream_name, stream_type).
    # Specify Processes: name, agent function, 
    #       lists of inputs and outputs and sources, additional arguments.
    multicore_specification = [
        # Streams
        [('x', 'f'), ('sines', 'f'), ('cosines', 'f'), ('tangents', 'f')],
        # Processes
        [{'name': 'sine', 'agent': sine, 'inputs':['x'], 'outputs': ['sines']},
         {'name': 'cosine', 'agent': cosine, 'inputs':['x'], 'outputs': ['cosines']},
         {'name': 'tanget', 'agent': tangent, 'inputs':['x'], 'outputs': ['tangents']},
         {'name': 'coordinator', 'agent': coordinate, 'inputs':['x', 'sines', 'cosines', 'tangents'],
          'sources': ['x']}]
    ]

    # Step 2: Create processes.
    processes, procs = get_processes_and_procs(multicore_specification)

    # Step 3: Create threads (if any)
    thread_0 = threading.Thread(target=source_thread_target, args=(procs,))

    # Step 4: Specify which process each thread runs in.
    # thread_0 runs in the process called 'coordinator'
    procs['coordinator'].threads = [thread_0]

    # Step 5: Start, join and terminate processes.
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

example_merging_streams_from_multiple_processes()

## Passing Data to and from Multiprocessing Blocks
Non-IoTPy processes or threads can interact concurrently with IoTPy by extending input streams, getting data from queues fed by output streams, and by putting data into, and getting data from, multiprocessing blocks.
![alt text](ExampleOfInteractionWithNonIoTPy.jpg "Title")
This example illustrates how to pass data to a multiprocessing block and get data from the block. This example is the same as the previous one except that the variables <b>total</b> and <b>num</b> are passed to the multiprocessing block which returns updated values of these variables.
<br>
<br>
total = multiprocessing.Value('f')
<br>
num = multiprocessing.Value('i')
<br>
<br>
creates <i>total</i> a wrapper for a float, and <i>num</i> a wrapper for int. 
<br>
<br>
These variables can be passed to any collection of processes. In this example they are passed only to the process <i>coordinator</i>.
These variables are assigned initial values from a computation that is not shown here. The multiprocessing block shown updates these values. For example, the value of <i>num</i> is 25 before the block is executed and 45 after it terminates.

### Passing variables as keyword or positional arguments
In this example, variables are passed to the process <i>coordinator</i> as keyword arguments.
The keyword arguments are specified as a dict with the name of an argument (e.g. 'total') and its initial value (<i>total</i>).
<br>
<br>
{'name': 'coordinator', 'agent': coordinate, 'inputs':['x', 'sines', 'cosines', 'tangents'],
<br>
 'sources': ['x'], 'source_functions':[sequence],
<br>
<b>'keyword_args'</b> : {'total' : total, 'num' : num},}

In [ ]:
import multiprocessing

def example_passing_data_to_multicore():
    total = multiprocessing.Value('f')
    num = multiprocessing.Value('i')
    # Values computed from an earlier computation which is not shown.
    # total and num are passed to the multiprocessing block.
    total.value = 4.0e-13
    num.value = 25

    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agent functions.

    # sine is the agent function for the process called  'sine'.
    def sine(in_streams, out_streams):
        map_element(np.sin, dtype_float(in_streams[0]), 
                    out_streams[0], name='sine')

    # cosine is the agent function for the process called 'cosine'.
    def cosine(in_streams, out_streams):
        map_element(np.cos, dtype_float(in_streams[0]), out_streams[0], name='cosine')

    # tangent is the agent function for the process called 'tangent'.
    def tangent(in_streams, out_streams):
        map_element(np.tan, dtype_float(in_streams[0]), 
                    out_streams[0], name='tangent')

    # coordinate is the agent function for the process called 'coordinate'.
    def coordinate(in_streams, out_streams, total, num):
        x, sines, cosines, tangents = in_streams

        def f(lst): return lst[0]/lst[1]

        def g(lst):
            error_squared= (lst[0] - lst[1])**2
            return error_squared
    
        ratios = Stream('ratios')
        errors = Stream('errors')
        zip_map(f, [sines, cosines], ratios, name='sine / cosine')
        zip_map(g, [ratios, tangents], errors, name='compute error')
        print_stream(errors, 'error')

    # Step 0.1: Define source thread target (if any).
    def source_thread_target(procs):
        extend_stream(procs, data=np.linspace(0.0, np.pi, 10), stream_name='x')
        terminate_stream(procs, stream_name='x')

    # Step 1: multicore_specification of streams and processes.
    # Specify Streams: list of pairs (stream_name, stream_type).
    # Specify Processes: name, agent function, 
    #       lists of inputs and outputs and sources, additional arguments.
    multicore_specification = [
        # Streams
        [('x', 'f'), ('sines', 'f'), ('cosines', 'f'), ('tangents', 'f')],
        # Processes
        [{'name': 'sine', 'agent': sine, 'inputs':['x'], 'outputs': ['sines']},
         {'name': 'cosine', 'agent': cosine, 'inputs':['x'], 'outputs': ['cosines']},
         {'name': 'tanget', 'agent': tangent, 'inputs':['x'], 'outputs': ['tangents']},
         {'name': 'coordinator', 'agent': coordinate, 'inputs':['x', 'sines', 'cosines', 'tangents'],
          'sources': ['x'], 'keyword_args' : {'total' : total, 'num' : num}}]
    ]

    # Step 2: Create processes.
    processes, procs = get_processes_and_procs(multicore_specification)

    # Step 3: Create threads (if any)
    thread_0 = threading.Thread(target=source_thread_target, args=(procs,))

    # Step 4: Specify which process each thread runs in.
    # thread_0 runs in the process called 'coordinator'
    procs['coordinator'].threads = [thread_0]

    # Step 5: Start, join and terminate processes.
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

example_passing_data_to_multicore()

## Actuators
A multiprocessing block may need to interact asynchronously with some external device. To do so, the block puts data into a queue and uses threads responsible for interfacing between the queue and the device. This simple example illustrates the simplest actuator: a printer. Indeed printing can be done synchronously by the multiprocessing block. Printing doesn't need a queue to interface between it and the block. We use the printer in this example to illustrate the idea.
<br>
<br>
Function <i>g</i> of process <i>p1</i> has an agent called 'copy_stream_s_to_queue_q' which  copies stream <i>s</i> to queue <i>q</i>. A thread, <b>my_thread</b> in <i>p1</i> prints values from the queue; this thread represents the thread that interfaces with an external actuator device. This thread is in addition to any source threads that may exist.
<br>
<br>
Queue <i>q</i> is specified as an <b>output queue</b>. An output queue gets a special message <b>'_finished'</b> when the multiprocess block terminates.
<br>
<br>
Threads (apart from source threads) and output queues are specified in <i>multicore_specifications</i>. See
<br>
<br>
{'name': 'p1', 'agent': g, 'inputs': ['y'], 
<br>
 'args': [q], <b>'output_queues'</b>: [q], <b>'threads'</b>: [my_thread]}
<br>
<br>
The thread, <i>my_thread</i>, terminates when it receives a '_finished' message. We want this thread to terminate so that process <i>p1</i> terminates, and then the entire multiprocessing block can terminate as well.
![alt text](ExamplesOfOutputQueue.jpg "Title")

In [ ]:
import threading
from IoTPy.agent_types.sink import stream_to_queue

def example_output_thread_with_queue():
    q = multiprocessing.Queue()

    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agent functions.

    # g is the agent function for the process called 'p1'.
    def g(in_streams, out_streams, q):
        s = Stream('s')
        map_element(lambda v: v*2, in_streams[0], s)
        stream_to_queue(s, q, name='copy_stream_s_to_queue_q')

    # Step 0.1: Define source thread target (if any).
    def source_thread_target(procs):
        for i in range(3):
            extend_stream(procs, data=list(range(i*2, (i+1)*2)), 
                          stream_name='x')
            time.sleep(0.001)
        terminate_stream(procs, stream_name='x')

    # Define the actuator thread target. This thread target is
    # used to create a thread (output) which is run in the process
    # called 'p0'.
    def get_data_from_output_queue(q):
        while True:
            v = q.get()
            if v == '_finished': break
            else: print ('q.get() = ', v)

    multicore_specification = [
        # Streams
        [('x', 'i'), ('y', 'i')],
        # Processes
        [{'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y'], 'sources': ['x']},
         {'name': 'p1', 'agent': g, 'inputs': ['y'], 
          'args': [q], 'output_queues': [q]}]
    ]

    processes, procs = get_processes_and_procs(multicore_specification)
    source_thread = threading.Thread(target=source_thread_target, args=(procs,))
    output_thread = threading.Thread(target=get_data_from_output_queue, args=(q,))
    procs['p0'].threads = [source_thread, output_thread]

    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

example_output_thread_with_queue()

## Example of Process Structure with Feedback
The example shows a process structure with feedback. This example creates an echo from a spoken sound. (You can write more efficient and succinct code to compute echoes. The code in this example is here merely because it illustrates a concept.)
![alt text](ExamplesOfFeedback.jpg "Title")
<br>
### Streams
<ol>
    <li><b>sound_made</b>: This is the sound made by a speaker in a large spherical space.</li>
    <li><b>attenuated</b>: This is the sound made multiplied by an attenuation factor.</li>
    <li><b>echo</b>: This is the echo of the sound made heard at the center of the room. The echo is a delay followed by an attenuation of the sound heard. </li>
    <li><b>sound_heard</b>: This is the sound that is heard by the speaker. The heard sound is the sound made by the speaker plus the echo.</li>
</ol>
The equations that define the streams are:
<ol>
    <li>
    <b>attentuated[n] = sound_heard[n]*attenuation</b>
    </li>
    <li>
    <b>echo[n] = attentuated[n-delay]</b> for n > delay.
    </li>
    <li>
    <b>sound_heard[n] = sound_made[n] + echo[n]</b> for n > delay.
    </li>
</ol>

### Process Structure
Process <i>p0</i> has a source which feeds one of its input streams <i>sound_made</i> with a stream of measurements obtained from a microphone. In this example, the stream is generated with numbers so that we can see how streams are processed.
<br>
<br>
Process <i>p1</i> contains a single input stream which is the sound heard and a single output stream which is an attenuation of the sound heard. 

### Process Functions
The function <i>f</i> of <i>p0</i> computes <i>echo</i> from <i>sound_made</i>. The first 4 , i.e., <b>delay</b>, units of the echo are empty (i.e. 0). 
<br>
<b>map_element(lambda v: v, attenuated, echo)</b>
<br>
copies the attenuated stream to the echo stream; however, since the first 4 (i.e. delay) values of the echo stream are 0, the echo stream will consist of 4 zeroes followed by the attenuated stream.
<br>
<i>out_streams[0]</i> of process <i>p0</i> is <i>sound_heard</i>. Function <i>f</i> makes <i>sound_heard</i> the sum of the echo and the sound made.
<br>
The function <i>g</i> of process <i>p1</i> <i>p0</i> puts elements of its input stream (i.e. <i>sound_heard</i> on queue <i>q</i> and returns the elements multiplied by <i>attenuation</i>.

In [ ]:
from IoTPy.agent_types.basics import *

def example_echo_two_cores():
    # This is the delay from when the made sound hits a
    # reflecting surface.
    delay = 6

    # This is the attenuation of the reflected wave.
    attenuation = 0.5

    # The results are put in this queue. A thread reads this
    # queue and feeds a speaker or headphone.
    q = multiprocessing.Queue()

    # ----------------------------------------------
    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agent functions.
    
    # Agent function for process named 'p0'
    # echo is a delay of zeroes followed by attenuated heard sound.
    # out_streams[0], which is the same as sound_heard, is
    # echo + sound_made
    def f_echo(in_streams, out_streams, delay):
        sound_made, attenuated = in_streams
        echo = StreamArray('echo', dtype='float')
        echo.extend(np.zeros(delay, dtype='float'))
        map_element(lambda v: v, attenuated, echo)
        # The zip_map output is the sound heard which is
        # the sound heard plus the echo.
        zip_map(sum, [sound_made, echo], out_streams[0])

    # Agent function for process named 'p1'
    # This process puts the sound heard into the output queue
    # and returns an attenuated version of the sound_heard as 
    # its output stream.
    def g_echo(in_streams, out_streams, attenuation, q):
        def gg(v):
            # v is the sound heard
            q.put(v)
            # v*attenuation is the echo
            print ('in g_echo; v is ', v)
            return v*attenuation
        map_element(gg, in_streams[0], out_streams[0])

    def source_thread_target(procs):
        data=list(range(10))
        extend_stream(procs, data=np.array(np.arange(10.0)), stream_name='sound_made')
        time.sleep(0.0001)
        extend_stream(procs, data=np.array([0.0]*10), stream_name='sound_made')
        terminate_stream(procs, stream_name='sound_made')

    # Thread that gets data from the output queue
    # This thread is included in 'threads' in the specification.
    # Thread target
    def get_data_from_output_queue(q):
        finished_getting_output = False
        while not finished_getting_output:
            v = q.get()
            if v == '_finished': break
            print ('heard sound = spoken + echo: ', v)

    multicore_specification = [
        # Streams
        [('sound_made', 'f'), ('attenuated', 'f'), ('sound_heard', 'f')],
        # Processes
        [{'name': 'p0', 'agent': f_echo, 'inputs': ['sound_made', 'attenuated'], 
          'outputs': ['sound_heard'], 'keyword_args' : {'delay' : delay}, 'sources': ['sound_made']},
         {'name': 'p1', 'agent': g_echo, 'inputs': ['sound_heard'], 'outputs': ['attenuated'],
          'args': [attenuation, q], 'output_queues': [q] } ]]

    processes, procs = get_processes_and_procs(multicore_specification)
     
    source_thread = threading.Thread(target=source_thread_target, args=(procs,))
    output_thread = threading.Thread(target=get_data_from_output_queue, args=(q,))
    procs['p0'].threads = [source_thread, output_thread]

    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

example_echo_two_cores()

## Example source and actuator thread with single process
This example is the same as the previous one except that the computation is carried out in a single process rather in two processes. The example illustrates an actuator thread and a source thread in the same process.
![alt text](ExamplesOfSingleProcess.jpg "Title")

In [ ]:
def example_echo_single_core():
    # This is the delay from when the made sound hits a
    # reflecting surface.
    delay = 4

    # This is the attenuation of the reflected wave.
    attenuation = 0.5

    # The results are put in this queue. A thread reads this
    # queue and feeds a speaker or headphone.
    q = multiprocessing.Queue()

    # ----------------------------------------------
    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agent functions.

    # Agent function for process named 'p0'
    # echo is a delay of zeroes followed by attenuated heard sound.
    # out_streams[0], which is the same as sound_heard is
    # echo + sound_made
        
    def f_echo(in_streams, out_streams, delay, attenuation, q):
        echo = StreamArray('echo', dtype='float')
        echo.extend(np.zeros(delay, dtype='float'))
        #echo = Stream('echo', initial_value=[0]*delay)
        #Note: sound_made = in_streams[0]
        sound_heard = in_streams[0] + echo
        map_element(lambda v: v*attenuation, sound_heard, echo)
        stream_to_queue(sound_heard, q)

    def source_thread_target(procs):
        extend_stream(procs, data=list(range(10)), stream_name='sound_made')
        time.sleep(0.0001)
        extend_stream(procs=procs, data=[0]*10, stream_name='sound_made')
        terminate_stream(procs, stream_name='sound_made')

    # Thread that gets data from the output queue
    # This thread is included in 'threads' in the specification.
    # Thread target
    def get_data_from_output_queue(q):
        finished_getting_output = False
        while not finished_getting_output:
            v = q.get()
            if v == '_finished': break
            print ('heard sound = spoken + echo: ', v)

    multicore_specification = [
        # Streams
        [('sound_made', 'f')],
        # Processes
        [{'name': 'p0', 'agent': f_echo, 'inputs': ['sound_made'],
          'args' : [delay, attenuation, q], 'sources': ['sound_made'],'output_queues': [q]}]]

    processes, procs = get_processes_and_procs(multicore_specification)
     
    source_thread = threading.Thread(target=source_thread_target, args=(procs,))
    output_thread = threading.Thread(target=get_data_from_output_queue, args=(q,))
    procs['p0'].threads = [source_thread, output_thread]

    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

example_echo_single_core()

## Example of a grid computation
Grid computations are used in science, for example in computing the temperature of a metal plate. The grid is partitioned into regions with a process assigned to simulate each region. On the n-th step, each process reads the values of relevant parts of the grid and updates its own value.
<br>
<br>
This example uses two copies of the grid; the two copies are <b>even</b> and <b>odd</b>. 
<ol>
    <li>On <b>even</b> steps (i.e., steps 0, 2, 4,..) the <i>j</i>-th
        proces <b>reads</b> the <i>even</i> grid and <b>writes</b> the
        <i>j</i>-th element of the <i>odd</i> grid. </li>
    <li>On <b>odd</b> steps, the <i>j</i>-th proces <b>reads</b> the
        <i>odd</i> grid and <b>writes</b> the <i>j</i>-th element of the 
        <i>even</i> grid. </li>
</ol>
So, each portion of the grid is modified by only one process. And no process reads a value while it is modified.

### The example problem
A linear metal bar of length <i>N</i> is partitioned into a grid of <i>N</i> continuous regions. Grid 0 is kept at a constant temperature of 0 degrees while grid <i>N-1</i> is kept at a constant temperature of <i>N-1</i> degrees. Initially, the temperature at intermediate grid points is arbitrary; in the code below, the temperature at grid point <i>i</i> exceeds <i>i</i> by <i>DELTA</i>.
<br>
<br>
Let <b>TEMP[i][k]</b> be the temperature of the <i>i</i>-th region on step <i>k</i>. Then, for all <i>k</i>:
<ol>
    <li>TEMP[0][k] = 0 </li>
    <li>TEMP[N-1][k] = N-1 </li>
    <li>TEMP[i][k] = (TEMP[i-1][k] + TEMP[i][k] + TEMP[i+1][k])/3 i in [1, ..,N-2] </li>
</ol>


### Processes
The computation uses <i>N-2</i> processes. The <i>i</i>-th process is called 'grid_i' and is responsible for simulating the <i>i</i>-th region.
<br>
Each process takes the <i>k + 1</i>-th step after it has finished the <i>k</i>-th step and it has determined that its neighbors have also finished the <i>k</i>-th step.
<br>


### Streams
The system has one stream, <b>s_i</b> for the <i>i</i>-th process. This stream contains the elements [0, 1, .. , k] after the <i>i</i>-th process has completed <i>k</i>-th steps.
<br>
Process <i>grid_i</i> outputs stream <i>s_i</i> and inputs streams from its neighboring processes which are <i>grid_(i-1)</i> if <i>i</i> exceeds 1 and <i>grid_(i+1)</i> if <i>i</i> is less than <i>N-1</i>.

### Process Structure
The process structure is linear with each process getting input streams from each of its neighbors and sending its output stream to all its neighbors.
![alt text](ExamplesOfGrid.jpg "Title")

### Process Function
The process begins by sending 0 on its output stream to indicate that it has finished its 0-th step.
<br>
<br>
The <i>k</i>-th value of <i>in_streams[j]</i> is <i>k</i> when the <i>j</i>-th neighboring process has completed its <i>k</i>-th step.
<br>
<br>
<b>synch_stream</b> is an internal stream of the process. The <i>k</i>-th element of this stream is <i>k</i> after all neighboring processes have completed their <i>k</i>-th step.
<br>
<br>
The zip_map function <i>r</i> operates on a list with one element from each neighbor. All the elements of the list will be <i>k</i> on the <i>k</i>-th step. The zip_map function returns <i>k</i> which is any element of the list. In this example it returns the 0-th element.
<br>
<br>
Thus the zip_map function acts as a synchronizer. It waits until all neighbors have completed the <i>k</i>-step and then it outputs <i>k</i>.
<br>
<br>
Function <i>g</i> is called for the <i>k</i>-th time when this process and all its neighbors have completed <i>k - 1</i> steps. Function <i>g</i> does the grid computation. Function <i>r</i> and the zip_map agent are used merely for synchronizing.
### run()
Function <i>f</i> calls <b>run</b> after it has declared all its agents. Without calling run() the function will take no action.
<br>
<br>
Note that when using external source threads, you should not call <i>run</i> because the source threads are responsible for starting and stopping the main computational thread. This example has no source threads so you must call <i>run</i> to start the system.

In [ ]:
from IoTPy.core.stream import _no_value
def test_grid():
    # N is the size of the grid
    N = 5
    # M is the number of steps of execution.
    M = 5
    # DELTA is the deviation from the final solution.
    DELTA = 0.01
    # even, odd are the grids that will be returned
    # by this computation
    even = multiprocessing.Array('f', N)
    odd = multiprocessing.Array('f', N)
    # Set up initial values of the grid.
    for i in range(1, N-1):
        even[i] = i + DELTA
    even[N-1] = N-1
    odd[N-1] = N-1
        
    def f(in_streams, out_streams, index, even, odd):
        def g(v):
            if (0 < index) and (index < N-1):
                if v%2 == 0:
                    odd[index] = (even[index-1] + even[index] + even[index+1])/3.0
                else:
                    even[index] = (odd[index-1] + odd[index] + odd[index+1])/3.0
            return v+1

        def r(lst, state):
            if state < M:
                return lst[0], state+1
            else:
                return _no_value, state
        for out_stream in out_streams: out_stream.extend([0])
        synch_stream = Stream('synch_stream')
        zip_map(r, in_streams, synch_stream, state=0, name='zip_map_'+str(index))
        map_element(g, synch_stream, out_streams[0], name='grid'+str(index))
        run()

    multicore_specification = [
        # Streams
        [('s_'+str(index), 'i') for index in range(1, N-1)],
        # Processes
        [{'name': 'grid_'+str(index), 'agent': f, 
          'inputs':['s_'+str(index+1), 's_'+str(index-1)], 
          'outputs':['s_'+str(index)], 
          'args': [index, even, odd]} for index in range(2, N-2)] + \
        [{'name': 'grid_'+str(1), 'agent': f, 
          'inputs':['s_'+str(2)], 'outputs':['s_'+str(1)], 
          'args': [1, even, odd]}] + \
        [{'name': 'grid_'+str(N-2), 'agent': f, 
          'inputs':['s_'+str(N-3)], 'outputs':['s_'+str(N-2)], 
          'args': [N-2, even, odd]}]
    ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

    print ('Grid after ', M, ' steps is: ')
    if M%2 == 0:
        print (even[:])
    else:
        print (odd[:])

test_grid()

# Counting Elements in Streams
The next example uses algorithms to count elements in streams (see IoTPy/examples/Counting). This example uses a Bloom Filter and a count-min-sketch algorithm. The data stream consists of pairs where a pair is either ('add', object) or ('check', object). The pair ('add', z) in the input stream states that object z was added to the data stream. The pair ('check', z) in the input stream is a command to check whether z was added earlier.
<br>
![alt text](CountingAlgorithms.jpg "Counting Elements in Streams")

In [ ]:
from examples.Counting.bloom_filter import bloom_filter_stream
from examples.Counting.bloom_filter import BloomFilter
from examples.Counting.count_min_sketch import count_min_sketch_stream
from examples.Counting.count_min_sketch import CountMinSketch
from IoTPy.agent_types.merge import merge_asynch

In [ ]:
def test_multiprocessing_counting_algorithms():
    # ----------------------------------------------
    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agents
    def bloom_filter_agent(in_streams, out_streams):
        bloom_filter = BloomFilter(
            est_elements=1000, false_positive_rate=0.05)
        bloom_filter_stream(in_streams[0], out_streams[0], 
                            bloom_filter=bloom_filter)

    def count_min_sketch_agent(in_streams, out_streams):
        count_min_sketch = CountMinSketch(width=1000, depth=20)
        count_min_sketch_stream(in_streams[0], out_streams[0],
                                count_min_sketch=count_min_sketch)

    def merge_agent(in_streams, out_streams):
        s = Stream('print stream')
        def g(pair):
            index, value = pair
            if index == 0:
                print ('bloom_filter. value: ', value)
            else:
                print ('count_min_sketch. value: ', value)
        merge_asynch(g, in_streams, s)

    # Step 0.1: Define source thread target (if any).
    def source_thread_target(procs):
        data=[('add', 'a'), ('add', 'b'), ('add', 'a'),
              ('check', 'c'), ('add', 'd'), ('check','a')]
        extend_stream(procs, data, stream_name='data')
        time.sleep(0.001)
        data=[('add', 'c'), ('check', 'b'), ('check', 'a'),
              ('check', 'c'), ('check', 'e'), ('add', 'a')]
        extend_stream(procs, data, stream_name='data')

        terminate_stream(procs, stream_name='data')
    
    # Step 1: multicore_specification of streams and processes.
    # Specify Streams: list of pairs (stream_name, stream_type).
    # Specify Processes: name, agent function, 
    #       lists of inputs and outputs and sources, additional arguments.
    multicore_specification = [
        # Streams
        [('data', 'x'), ('bloom_results', 'x'),
         ('count_min_sketch_results', 'x')],
        # Processes
        [{'name': 'bloom_filter_process', 'agent': bloom_filter_agent, 
          'inputs':['data'], 'outputs': ['bloom_results'],
          'sources': ['data']},
         {'name': 'count_min_sketch_process', 'agent': count_min_sketch_agent, 
          'inputs':['data'], 'outputs': ['count_min_sketch_results']},
         {'name': 'merge_process', 'agent': merge_agent,
          'inputs': ['bloom_results', 'count_min_sketch_results']}
        ]]

    # Step 2: Create processes.
    processes, procs = get_processes_and_procs(multicore_specification)

    # Step 3: Create threads (if any)
    thread_0 = threading.Thread(target=source_thread_target, args=(procs,))

    # Step 4: Specify which process each thread runs in.
    # thread_0 runs in the process called 'coordinator'
    procs['bloom_filter_process'].threads = [thread_0]

    # Step 5: Start, join and terminate processes.
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

In [ ]:
test_multiprocessing_counting_algorithms()